In [1]:
# Cell 1: Set the files you want to convert
# Just modify this list with the files you want

files_to_convert = [
    r"c:\Users\Moses\math_ops\OperatorKernelO6\OperatorKernelO6\Meta\Termination.lean",
    r"c:\Users\Moses\math_ops\OperatorKernelO6\core_docs\Termination_Companion.md",
    r"c:\Users\Moses\math_ops\OperatorKernelO6\core_docs\ko6_guide.md"
]

print(f"Ready to convert {len(files_to_convert)} files:")
for i, file in enumerate(files_to_convert, 1):
    print(f"{i}. {file}")

Ready to convert 3 files:
1. c:\Users\Moses\math_ops\OperatorKernelO6\OperatorKernelO6\Meta\Termination.lean
2. c:\Users\Moses\math_ops\OperatorKernelO6\core_docs\Termination_Companion.md
3. c:\Users\Moses\math_ops\OperatorKernelO6\core_docs\ko6_guide.md


In [ ]:
# Cell 2: Run the complete conversion (just hit enter on this cell)

import os
import subprocess
from pathlib import Path
from datetime import datetime
import re
import html

def get_file_info(filepath):
    """Get file information"""
    try:
        path = Path(filepath)
        if not path.exists():
            return None
        
        stat = path.stat()
        return {
            'name': path.name,
            'size': stat.st_size,
            'modified': datetime.fromtimestamp(stat.st_mtime).strftime('%Y-%m-%d %H:%M:%S')
        }
    except Exception as e:
        return {'error': str(e)}

def read_file_content(filepath):
    """Read file content with encoding detection"""
    try:
        path = Path(filepath)
        if not path.exists():
            return f"ERROR: File not found: {filepath}"
        
        # Try different encodings
        encodings = ['utf-8', 'utf-8-sig', 'cp1252', 'latin1']
        
        for encoding in encodings:
            try:
                with open(path, 'r', encoding=encoding) as f:
                    return f.read()
            except UnicodeDecodeError:
                continue
        
        return f"ERROR: Could not decode file with any encoding: {filepath}"
    except Exception as e:
        return f"ERROR: {str(e)}"

def process_markdown_content(content):
    """Convert markdown to HTML"""
    # Escape HTML first
    content = html.escape(content)
    
    # Headers
    content = re.sub(r'^### (.*?)$', r'<h3>\1</h3>', content, flags=re.MULTILINE)
    content = re.sub(r'^## (.*?)$', r'<h2>\1</h2>', content, flags=re.MULTILINE)
    content = re.sub(r'^# (.*?)$', r'<h1>\1</h1>', content, flags=re.MULTILINE)
    
    # Code blocks (before inline code)
    content = re.sub(r'```(\w+)?\n(.*?)\n```', 
                    r'<pre class="code-block"><code>\2</code></pre>', 
                    content, flags=re.DOTALL)
    
    # Inline code
    content = re.sub(r'`([^`]+)`', r'<code class="inline-code">\1</code>', content)
    
    # Bold and italic
    content = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', content)
    content = re.sub(r'\*(.*?)\*', r'<em>\1</em>', content)
    
    # Links
    content = re.sub(r'\[([^\]]+)\]\(([^)]+)\)', r'<a href="\2">\1</a>', content)
    
    # Lists
    lines = content.split('\n')
    processed_lines = []
    in_list = False
    
    for line in lines:
        if re.match(r'^\s*[-*+]\s+', line):
            if not in_list:
                processed_lines.append('<ul>')
                in_list = True
            item = re.sub(r'^\s*[-*+]\s+', '', line)
            processed_lines.append(f'<li>{item}</li>')
        else:
            if in_list:
                processed_lines.append('</ul>')
                in_list = False
            processed_lines.append(line)
    
    if in_list:
        processed_lines.append('</ul>')
    
    content = '\n'.join(processed_lines)
    
    # Convert line breaks to paragraphs
    paragraphs = content.split('\n\n')
    processed_paragraphs = []
    
    for para in paragraphs:
        para = para.strip()
        if para and not para.startswith('<') and not para.endswith('>'):
            # Don't wrap already tagged content
            if not re.match(r'^<(h[1-6]|ul|li|pre)', para):
                para = f'<p>{para}</p>'
        processed_paragraphs.append(para)
    
    return '\n'.join(processed_paragraphs)

def create_html_output(filepath, content, file_info):
    """Create HTML output for a file"""
    
    # Determine file type and processing
    ext = Path(filepath).suffix.lower()
    
    if ext == '.md':
        # Process markdown to HTML
        processed_content = process_markdown_content(content)
        content_section = f'<div class="markdown-content">{processed_content}</div>'
        language = 'markdown'
    else:
        # For code files, escape HTML and wrap in pre/code
        escaped_content = html.escape(content)
        content_section = f'<pre class="code-content"><code>{escaped_content}</code></pre>'
        language = 'lean' if ext == '.lean' else 'text'
    
    html_template = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{file_info.get('name', 'Document')}</title>
    <style>
        body {{
            width: 100%;
            max-width: 1200px;
            margin: 0 auto;
            padding: 30px;
            font-size: 14px;
            line-height: 1.6;
            color: #333;
            background: #fff;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }}
        
        h1 {{
            color: #2c3e50;
            border-bottom: 3px solid #3498db;
            padding-bottom: 15px;
            margin-bottom: 30px;
            font-size: 28px;
        }}
        
        h2 {{
            color: #34495e;
            border-bottom: 2px solid #bdc3c7;
            padding-bottom: 10px;
            margin-top: 40px;
            margin-bottom: 20px;
            font-size: 22px;
        }}
        
        h3 {{
            color: #34495e;
            margin-top: 30px;
            margin-bottom: 15px;
            font-size: 18px;
        }}
        
        .metadata {{
            background: #e8f4fd;
            padding: 20px;
            border-radius: 8px;
            border-left: 4px solid #3498db;
            margin: 25px 0;
            font-size: 13px;
        }}
        
        .code-content {{
            background: #f8f9fa;
            padding: 25px;
            border-radius: 8px;
            overflow-x: auto;
            border: 1px solid #dee2e6;
            font-size: 13px;
            line-height: 1.5;
            white-space: pre-wrap;
            word-wrap: break-word;
            font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
        }}
        
        .code-block {{
            background: #f8f9fa;
            padding: 20px;
            border-radius: 6px;
            overflow-x: auto;
            border: 1px solid #dee2e6;
            margin: 20px 0;
            font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
            font-size: 13px;
            line-height: 1.4;
        }}
        
        .inline-code {{
            font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
            background: #f1f3f4;
            padding: 3px 6px;
            border-radius: 3px;
            font-size: 90%;
            color: #d73a49;
        }}
        
        .markdown-content {{
            line-height: 1.7;
        }}
        
        .markdown-content p {{
            margin: 15px 0;
        }}
        
        .markdown-content ul {{
            margin: 15px 0;
            padding-left: 25px;
        }}
        
        .markdown-content li {{
            margin: 8px 0;
        }}
        
        .markdown-content strong {{
            font-weight: 600;
            color: #2c3e50;
        }}
        
        .markdown-content em {{
            font-style: italic;
            color: #555;
        }}
        
        .markdown-content a {{
            color: #3498db;
            text-decoration: none;
        }}
        
        .markdown-content a:hover {{
            text-decoration: underline;
        }}
        
        .print-instructions {{
            background: #fff3cd;
            padding: 20px;
            border-radius: 8px;
            border-left: 4px solid #ffc107;
            margin-top: 40px;
            page-break-inside: avoid;
        }}
        
        @media print {{
            body {{ 
                width: 100%; 
                max-width: none; 
                margin: 0; 
                padding: 20px; 
                font-size: 12px;
                line-height: 1.4;
            }}
            .code-content {{ 
                font-size: 11px; 
                line-height: 1.3; 
                margin: 15px 0; 
                padding: 20px;
                background: #f8f9fa !important;
                border: 1px solid #dee2e6 !important;
                page-break-inside: avoid;
                overflow: visible;
            }}
            .code-block {{ 
                font-size: 11px; 
                line-height: 1.3; 
                padding: 15px;
                page-break-inside: avoid;
            }}
            .print-instructions {{ 
                display: none; 
            }}
            h1 {{ 
                font-size: 20px; 
                page-break-after: avoid;
                margin-bottom: 20px;
            }}
            h2 {{ 
                font-size: 16px; 
                page-break-after: avoid;
                margin-top: 25px;
            }}
            h3 {{ 
                font-size: 14px; 
                page-break-after: avoid;
            }}
            .metadata {{ 
                margin: 15px 0; 
                padding: 15px;
                page-break-inside: avoid;
            }}
        }}
        
        @page {{ 
            margin: 0.7in; 
            size: letter; 
        }}
    </style>
</head>
<body>
    <h1>{file_info.get('name', 'Document')}</h1>
    
    <div class="metadata">
        <p>
            <strong>File:</strong> <code>{filepath}</code><br>
            <strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}<br>
            <strong>File Size:</strong> {file_info.get('size', 'Unknown')} bytes<br>
            <strong>Last Modified:</strong> {file_info.get('modified', 'Unknown')}
        </p>
    </div>
    
    {content_section}
    
    <div class="print-instructions">
        <h3>Print to PDF Instructions</h3>
        <ol>
            <li>Press <strong>Ctrl+P</strong> to open print dialog</li>
            <li>Choose "Save as PDF" or "Microsoft Print to PDF"</li>
            <li>Set margins to "Normal" (0.7 inch)</li>
            <li>Enable "Background graphics" to preserve colors</li>
            <li>Choose "More settings" → "Paper size" → "Letter"</li>
        </ol>
    </div>
</body>
</html>"""
    
    return html_template

def convert_files():
    """Convert all files to HTML"""
    output_dir = Path("output_docs")
    output_dir.mkdir(exist_ok=True)
    
    results = []
    
    for filepath in files_to_convert:
        print(f"\\nProcessing: {filepath}")
        
        # Get file info
        file_info = get_file_info(filepath)
        if file_info is None:
            print(f"  ❌ File not found: {filepath}")
            results.append(('FAILED', filepath, 'File not found'))
            continue
        
        if 'error' in file_info:
            print(f"  ❌ Error: {file_info['error']}")
            results.append(('FAILED', filepath, file_info['error']))
            continue
        
        # Read content
        content = read_file_content(filepath)
        if content.startswith('ERROR:'):
            print(f"  ❌ {content}")
            results.append(('FAILED', filepath, content))
            continue
        
        # Create HTML
        html_content = create_html_output(filepath, content, file_info)
        
        # Save HTML file
        output_filename = Path(filepath).stem + '.html'
        output_path = output_dir / output_filename
        
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(html_content)
            
            print(f"  ✅ Created: {output_path}")
            results.append(('SUCCESS', filepath, str(output_path)))
            
        except Exception as e:
            print(f"  ❌ Failed to write: {e}")
            results.append(('FAILED', filepath, str(e)))
    
    return results

# Run the conversion
print("🚀 Starting file conversion...")
results = convert_files()

# Summary
print(f"\\n" + "="*60)
print("CONVERSION SUMMARY")
print("="*60)

success_count = sum(1 for r in results if r[0] == 'SUCCESS')
failed_count = len(results) - success_count

print(f"✅ Successful: {success_count}")
print(f"❌ Failed: {failed_count}")
print(f"📁 Output directory: {Path('output_docs').absolute()}")

if success_count > 0:
    print(f"\\n📄 Generated HTML files:")
    for status, source, output in results:
        if status == 'SUCCESS':
            print(f"   • {Path(output).name}")

if failed_count > 0:
    print(f"\\n⚠️  Failed files:")
    for status, source, error in results:
        if status == 'FAILED':
            print(f"   • {Path(source).name}: {error}")

print(f"\\n🎉 Done! Open the HTML files in your browser and press Ctrl+P to save as PDF.")

🚀 Starting file conversion...
\nProcessing: c:\Users\Moses\math_ops\OperatorKernelO6\OperatorKernelO6\Meta\Termination.lean
  ✅ Created: output_docs\Termination.html
\nProcessing: c:\Users\Moses\math_ops\OperatorKernelO6\core_docs\Termination_Companion.md
  ✅ Created: output_docs\Termination_Companion.html
\nProcessing: c:\Users\Moses\math_ops\OperatorKernelO6\core_docs\ko6_guide.md
  ✅ Created: output_docs\ko6_guide.html
\n============================================================
CONVERSION SUMMARY
✅ Successful: 3
❌ Failed: 0
📁 Output directory: c:\Users\Moses\math_ops\OperatorKernelO6\output_docs
\n📄 Generated HTML files:
   • Termination.html
   • Termination_Companion.html
   • ko6_guide.html
\n🎉 Done! Open the HTML files in your browser and press Ctrl+P to save as PDF.
